## set running directory to project source

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.utils.code_scraps import add_indy_sweep_tool, \
        use_current_place_point_only, use_current_sub_binders_only, \
        finish_L_shape, set_l_shape_object, ModeSwitcher, double_sweep_motions

from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene
from pkg.planning.pipeline import PlanningPipeline
from pkg.ui.ui_broker import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask, AbstractObject
from pkg.planning.constraint.constraint_subject import SweepLineTask
from pkg.planning.constraint.constraint_subject import SweepFrame
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT
from pkg.planning.constraint.constraint_common import sample_redundancy, combine_redundancy
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
import matplotlib.pyplot as plt
import math
import numpy as np
from collections import defaultdict
from demo_utils.area_select import DATASET_DIR, SweepDirections
try_mkdir(DATASET_DIR)

## create scene builder

In [ ]:
ROBOT_TYPE = RobotType.indy7
INDY_BASE_OFFSET = (0.172,0,0.439)
TIMEOUT_MP = 0.5

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                None)
robot_config = RobotConfig(1, ROBOT_TYPE, (INDY_BASE_OFFSET, (0,0,0)),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True})

crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config], connection_list=[False, False])

ROBOT_NAME = robot_config.get_indexed_name()
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

## make geometry scene

In [ ]:
BASE_LINK = "base_link"
EE_NAME = "ee_point"
gscene = s_builder.create_gscene(crob, start_rviz=True)

floor = gscene.create_safe(GEOTYPE.BOX, "floor", BASE_LINK, (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
# ee_point.draw_traj_coords([[0]*6])

## init planning pipeline

In [ ]:
from demo_utils.environment import add_kiro_indytool_down

pscene = PlanningScene(gscene, combined_robot=crob)
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
add_kiro_indytool_down(gscene, zoff=0, tool_link=TIP_LINK, face_name="dummy_brush")
ee_point = gscene.create_safe(GEOTYPE.SPHERE, EE_NAME, TIP_LINK, (0.01,)*3, 
                              center=(0,0,0), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.8), display=True, fixed=True, collision=False)

ppline = PlanningPipeline(pscene)

mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.update_gscene()
ppline.set_motion_planner(mplan)


# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

## Set Parameters

In [ ]:
TIMEOUT_MP = 0.5


VISUALIZE = False
VERBOSE = False
N_try = 3
SWEEP_DIV = 7


HOME_POSE_EXT = np.array([0,]*6 + np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0]).tolist())
crob.home_pose = HOME_POSE_EXT
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
gscene.show_pose(crob.home_pose)

# # dummy tool coord
# gscene.add_highlight_axis("tool", "coord", link_name="indy0_tcp", 
#                           center=(0.1,0,0.2), orientation_mat=Rot_rpy((0,0,np.pi)))

## Scan

In [ ]:
# HOME_POSE = np.zeros(gscene.joint_num)
HOME_POSE = HOME_POSE_EXT
for tip_dir, SWEEP_AXIS in [(SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    SWEEP_AX = "XYZ".index(SWEEP_AXIS)
    STEP_SIZES = [0.02]*3
    STEP_SIZES[SWEEP_AX] = 0.01
    X_step, Y_step, Z_step = STEP_SIZES

    if tip_dir == SweepDirections.front:
        X_range = (0.0, 1)
        Y_range = (-0.8, 0.8)
        Z_range = (0.0, 1.0)
    elif tip_dir == SweepDirections.up:
        X_range = (0.2, 0.8)
        Y_range = (-0.8, 0.8)
        Z_range = (-0.1, 1.3)
    elif tip_dir == SweepDirections.down:
        X_range = (0.2, 0.8)
        Y_range = (-0.8, 0.8)
        Z_range = (-0.7, 0.7)
    else:
        raise(NotImplementedError("SweepDirection not implemented"))
        
    DATASET_PATH = os.path.join(DATASET_DIR, SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS))
    R_be, STEP_AX, LEVEL_AX = SweepDirections.get_Re_step_level(tip_dir, SWEEP_AX)
        
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    X_DIR, Y_DIR, Z_DIR = np.identity(3, dtype=np.int)
    SWEEP_DIR = [X_DIR, Y_DIR, Z_DIR][SWEEP_AX]
    RANGE_STEPS_REF = [(X_range, X_step, X_DIR), (Y_range, Y_step, Y_DIR), (Z_range, Z_step, Z_DIR)]
    RANGE_STEPS = np.array(RANGE_STEPS_REF)[np.where(np.logical_not(SWEEP_DIR))]
    RANGEX, STEPX, DIRX = RANGE_STEPS_REF[SWEEP_AX]
    STEP_SWEEP = float(RANGEX[1] - RANGEX[0])/SWEEP_DIV
    
    RANGE1, STEP1, DIR1 = RANGE_STEPS[LEVEL_AX]
    RANGE2, STEP2, DIR2 = RANGE_STEPS[STEP_AX]
        

    range_list_dict = {}
    config_list_dict = {}
    STEPS_1 = np.round(np.arange(RANGE1[0], RANGE1[1]+STEP1, STEP1), 4)
    STEPS_2 = np.round(np.arange(RANGE2[0], RANGE2[1]+STEP2, STEP2), 4)
    STEPS_X = np.round(np.arange(RANGEX[0], RANGEX[1]-1e-3, STEP_SWEEP)[1:].tolist(), 4)
    LEN_1, LEN_2 = len(STEPS_1), len(STEPS_2)
    count = 0
    LEN_ALL = LEN_1*LEN_2
    gtimer.tic("test")
    for i_p1, pos1 in enumerate(STEPS_1):
        for i_p2, pos2 in enumerate(STEPS_2):
            count += 1
            with gtimer.block("loop"):
                pos_rest = np.multiply(pos1, DIR1) + np.multiply(pos2, DIR2)
                pos0 = pos_rest + np.multiply(np.mean(RANGEX), DIRX)
                key = tuple(np.round(pos0, 4))
                print("========= {} ({}/{})=========".format(key, count, LEN_ALL))
                T_be = SE3(R_be, pos0)
                T_bp = np.matmul(T_be, SE3(np.identity(3), (0.34,0,0.25)))
                gscene.create_safe(GEOTYPE.BOX, "sweep_surf", ROBOT_BASE, 
                                   dims = np.multiply(RANGEX[1]-RANGEX[0], DIRX) + np.multiply(0.01, DIR1) + np.multiply(0.4, DIR2), 
                                   center=T_bp[:3,3], rpy=(0,0,0), 
                                   color=(0.8,0.0,0.0,0.5), display=True,  
                                   fixed=True, collision=True)
                gscene.update_markers_all()
                if VISUALIZE:
                    gscene.add_highlight_axis("hl", "tbe0", link_name=ROBOT_BASE, 
                                              center=T_be[:3,3], orientation_mat=T_be[:3,:3], dims=[0.2,0.04,0.04])
                
                p2p_list = []
                qpair_list = []
                T_be0 = np.copy(T_be)
                for i_try in range(N_try):
                    traj, success = mplan.planner.plan_py(
                        ROBOT_NAME, TIP_LINK, np.concatenate(T2xyzquat(T_be0)), ROBOT_BASE, HOME_POSE, 
                        timeout=TIMEOUT_MP)
                    if not success:
                        continue
                    for i_px, posx in enumerate(STEPS_X):
                        pos_sweep = np.multiply(posx, DIRX)
                        pos = pos_rest + pos_sweep
                        T_be = SE3(R_be, pos)
                        if i_px > 0: # already done for 0
                            if VISUALIZE:
                                gscene.add_highlight_axis("hl", "tbe0", link_name=ROBOT_BASE, 
                                                          center=T_be[:3,3], orientation_mat=T_be[:3,:3])
                            traj, success = mplan.planner.plan_py(
                                ROBOT_NAME, TIP_LINK, np.concatenate(T2xyzquat(T_be)), ROBOT_BASE, HOME_POSE, 
                                timeout=TIMEOUT_MP)
                        if not success:
                            continue
                        Q0 = traj[-1]
                        if VISUALIZE:
                            gscene.show_pose(Q0)

                        pos_tar = np.multiply(RANGEX[0], DIRX)
                        dP_tar = pos_tar - pos_sweep
                        if VISUALIZE:
                            gscene.add_highlight_axis("hl", "tbe_n", link_name=ROBOT_BASE, 
                                                      center=T_be[:3,3]+dP_tar, orientation_mat=T_be[:3,:3])
                        if np.linalg.norm(dP_tar) <= 1e-6:
                            Qdown = Q0
                        else:
                            Traj_down, success = get_sweep_traj(mplan, ee_point, dP_tar, Q0, DP=STEPX, ERROR_CUT=0.01, 
                                                                ref_link=ROBOT_BASE, VISUALIZE=VISUALIZE, VERBOSE=VERBOSE)
                            if len(Traj_down)==0:
                                continue
                            Qdown = Traj_down[-1]

                        pos_tar = np.multiply(RANGEX[1], DIRX)
                        dP_tar = pos_tar - pos_sweep
                        if VISUALIZE:
                            gscene.add_highlight_axis("hl", "tbe_p", link_name=ROBOT_BASE, 
                                                      center=T_be[:3,3]+dP_tar, orientation_mat=T_be[:3,:3])
                        if np.linalg.norm(dP_tar) <= 1e-6:
                            Traj_up = Q0
                        else:
                            Traj_up, success = get_sweep_traj(mplan, ee_point, dP_tar, Q0, DP=STEPX, ERROR_CUT=0.01, 
                                                              ref_link=ROBOT_BASE, VISUALIZE=VISUALIZE, VERBOSE=VERBOSE)
                            if len(Traj_up)==0:
                                continue
                            Qup = Traj_up[-1]

                        p0 = ee_point.get_tf(list2dict(Q0, gscene.joint_names), 
                                             from_link=ROBOT_BASE)[SWEEP_AX,3]
                        pdown = ee_point.get_tf(list2dict(Qdown, gscene.joint_names), 
                                             from_link=ROBOT_BASE)[SWEEP_AX,3]
                        pup = ee_point.get_tf(list2dict(Qup, gscene.joint_names), 
                                             from_link=ROBOT_BASE)[SWEEP_AX,3]
                        p2p = (pdown, pup)
                        p2p0 = (pdown-p0, pup-p0)
                        qpair = (Qdown, Q0, Qup)
                        if np.multiply(*np.sign(p2p0))<0:
                            p2p_list.append(p2p)
                            qpair_list.append(qpair)

                range_list_dict[key] = np.array(p2p_list)
                config_list_dict[key] = np.array(qpair_list)
                if VISUALIZE:
                    gscene.clear_highlight()
        elapsed = gtimer.toc("test")/1000
        print(gtimer)
        print("=============================================")
        print("=============== {} / {} s===================".format(int(elapsed), int(elapsed/count*LEN_ALL)))

        print("=============================================")

    best_range_dict = {}
    for k,v in range_list_dict.items():
        if len(v)>0:
            i_max = np.argmax(v[:,1]-v[:,0])
            best_range_dict[k] = v[i_max]
        else:
            best_range_dict[k] = (0,0)
    np.save(DATASET_PATH, {"best_range_dict": best_range_dict, "range_list_dict": range_list_dict, "config_list_dict": config_list_dict})

## plot ranges

In [ ]:
plt.figure(figsize=(15,15*LEN_1))
for i_h, val_1 in enumerate(STEPS_1):
    plt.subplot(LEN_1, 1, i_h+1)
    plt.title(val_1)
    for k,v in range_list_dict.items():
        if round(np.dot(k, DIR1), 4)==round(val_1, 4):
            kk = np.dot(k, DIR2)
            for vv in v:
                plt.plot([kk+0.02, kk+0.02], vv,  '-o', linewidth=1)
            plt.plot([kk, kk], best_range_dict[k], '-ks', linewidth=3)
#     plt.axis([-1,1,0,1])
    plt.grid()

## Read and save min/max csv

In [ ]:
for tip_dir, SWEEP_AXIS in [(SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")]:
    filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)
    DATASET_PATH = os.path.join(DATASET_DIR, filename+".npy")
    dataset_read = np.reshape(np.load(DATASET_PATH, allow_pickle=True), (1,))[0]
    range_list_dict = dataset_read['range_list_dict']
    best_range_dict = dataset_read['best_range_dict']
    
    SWEEP_AXIS = "XYZ".index(SWEEP_AXIS)
    SWEEP_VEC = [0,0,0]
    SWEEP_VEC[SWEEP_AXIS] = 1
    R_be = SweepDirections.get_dcm_re(tip_dir)
    LEVEL_VEC = np.abs(R_be[:3,0]).astype(np.int) # Assum level along tip-link X axis
    STEP_VEC = np.abs(R_be[:3,1]).astype(np.int)  # Assum step along tip-link Y axis
    assert np.all(np.abs(R_be[:3,2]).astype(int) == SWEEP_VEC), "Sweep direction mismatch"
    LEVEL_AXIS = np.where(LEVEL_VEC)[0][0]
    STEP_AXIS = np.where(STEP_VEC)[0][0]
    min_points = []
    max_points = []
    for k,v in best_range_dict.items():
        min_points.append(tuple(np.add(k, np.multiply(v[0], SWEEP_VEC))))
        max_points.append(tuple(np.add(k, np.multiply(v[1], SWEEP_VEC))))
    min_points = sorted(min_points, key=lambda x: (x[LEVEL_AXIS], x[STEP_AXIS], x[SWEEP_AXIS]))
    max_points = sorted(max_points, key=lambda x: (x[LEVEL_AXIS], x[STEP_AXIS], x[SWEEP_AXIS]))
    for min_p, max_p in zip(min_points, max_points):
        assert min_p[LEVEL_AXIS] == max_p[LEVEL_AXIS], "Level value mismatch"
        assert min_p[STEP_AXIS] == max_p[STEP_AXIS], "Step value mismatch"
    np.savetxt(os.path.join(DATASET_DIR, filename+'-min.csv'), min_points, delimiter=",")
    np.savetxt(os.path.join(DATASET_DIR, filename+'-max.csv'), max_points, delimiter=",")

In [ ]:
np.add(k, np.multiply(v[1], SWEEP_VEC))

In [ ]:
best_range_dict

In [ ]:
gscene.clear_highlight()

## Show saved ranges

In [ ]:
tip_dir, SWEEP_AXIS = [(SweepDirections.front, "X"), (SweepDirections.up, "Z"), (SweepDirections.down, "Z")][2]
filename = SweepDirections.get_file_name(ROBOT_TYPE, tip_dir.name+SWEEP_AXIS)
DATASET_PATH = os.path.join(DATASET_DIR, filename+".npy")
sweep_path = os.path.join(DATASET_DIR, filename)
sweep_max = np.loadtxt(sweep_path + "-max.csv", delimiter=",")
sweep_min = np.loadtxt(sweep_path + "-min.csv", delimiter=",")
lines = [(p_min, p_max) for (p_min, p_max) in zip(sweep_min, sweep_max)]

In [ ]:
from demo_utils.area_select import show_lines
gscene.clear_highlight()
show_lines(gscene, lines, base_link=ROBOT_BASE, sweep_axis=SWEEP_AXIS.lower())

In [ ]:
lines_srt = sorted(lines, key=lambda x: np.linalg.norm(np.subtract(*x)))

In [ ]:
lines_srt[-1]

In [ ]:
R_be = SweepDirections.get_dcm_re(tip_dir)
T_be = SE3(R_be, pos0)

In [ ]:

traj, success = mplan.planner.plan_py(
    ROBOT_NAME, TIP_LINK, np.concatenate(T2xyzquat(T_be)), ROBOT_BASE, HOME_POSE, 
    timeout=TIMEOUT_MP)

In [ ]:
# DATASET_PATH_BAK1 = os.path.join(DATASET_DIR+"_bak", ROBOT_TYPE.name+'-up.npy')
# DATASET_PATH_BAK2 = os.path.join(DATASET_DIR+"_bak2", ROBOT_TYPE.name+'-up.npy')
# dataset_read_bak1 = np.reshape(np.load(DATASET_PATH_BAK1, allow_pickle=True), (1,))[0]
# dataset_read_bak2 = np.reshape(np.load(DATASET_PATH_BAK2, allow_pickle=True), (1,))[0]
# for k, v in dataset_read_bak1.items():
#     v.update(dataset_read_bak2[k])
# np.save(DATASET_PATH, dataset_read_bak1)